In [4]:
pip install pandas

  Created wheel for pd: filename=pd-0.0.1-py3-none-any.whl size=6539 sha256=b9f8dface4ec56ebf4a3aa223997bfd5a7683a2a75bf926841fc2d9975d8c619
  Stored in directory: c:\users\connor\appdata\local\pip\cache\wheels\c0\45\39\f9063988f991ef39fdbe4e5f2c0e9085e3e0ab0465602e33da
Successfully built pd
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install bokeh

  Created wheel for bokeh: filename=bokeh-2.3.2-py3-none-any.whl size=11334277 sha256=bcf8b4d7899aae056114fe7a2564349a71b50d86ec6c8c096e3b41f2b26f73f8
  Stored in directory: c:\users\connor\appdata\local\pip\cache\wheels\71\96\e1\80f7dbc16a958ed861b3c3ccd00b4c3b11951ff004856c398a
Successfully built bokeh
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DatePicker, Select
from bokeh.io import curdoc

import json
from datetime import datetime, timedelta

In [2]:
with open('./data.json') as f:
    d = json.load(f)
    d = d[list(d.keys())[0]]
    d = list(map(lambda x: json.loads(x['data']), d))
   
df = pd.json_normalize(d)
df['time'] = pd.to_datetime(df['time'], unit="ms")
time_series = df.groupby('time')['currentBpm'].mean()

In [23]:
def makeGraph(year, month, day, is_day=True):
    p1 = figure(x_axis_type="datetime", title="BPM", plot_width=1250, plot_height=500)
    p1.xaxis.axis_label = 'Date'
    p1.yaxis.axis_label = 'BPM'
    init_datetime = datetime(year,month,day)
    # Show Day
    if is_day:
        day_data = time_series[init_datetime:init_datetime+timedelta(days=1)]
        source = ColumnDataSource(pd.DataFrame(day_data))
        day_line = p1.line(x="time", y="currentBpm",source=source)
        p1.xaxis.ticker.desired_num_ticks = 24
    
    # Show Week
    else:
        start_of_week = init_datetime - timedelta(days=init_datetime.weekday())
        end_of_week = start_of_week + timedelta(days=6)
        cur_day = start_of_week
        multi_line_data_time = []
        multi_line_data_bpm = []
        while cur_day <= end_of_week:
            day_data = time_series[cur_day:cur_day+timedelta(days=1)]
            multi_line_data_time.append(day_data.index)
            multi_line_data_bpm.append(day_data.values)
            #multi_line_data.append(day_data)
            cur_day = cur_day + timedelta(days=1)
        source = ColumnDataSource(dict(time=multi_line_data_time, currentBpm=multi_line_data_bpm))
        week_line = p1.multi_line(xs="time", ys="currentBpm",source=source)
        p1.xaxis.ticker.desired_num_ticks = 6
    show(p1)

makeGraph(2021,4,21, is_day=True)


In [ ]:
# https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html